In [310]:
from PIL import Image
from colormath.color_objects import LabColor, sRGBColor, AdobeRGBColor
from colormath.color_conversions import convert_color
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [332]:
class pre_processing():
    def __init__(self,name):
        self.name = name
        self.image = Image.open(self.name)
        self.size = self.image.size
        self.processing()
    
    def processing(self):
        if (self.grayscale):
            self.cut()
            self.resize()
            self.save('resized_')
            self.rgb_image2lab_ndarray()
    
    def show_image(self):
        plt.imshow(self.image)
        plt.xticks([])
        plt.yticks([])
        
    def cut(self):
        size = min(self.size)
        self.image = self.image.crop((0,0,size,size))
    
    def resize(self):
        self.image = self.image.resize((128,128),Image.ANTIALIAS)
    
    def save(self,info):
        self.image.save(info+self.name+'.jpeg')
    
    def grayscale(self):
        return (self.image.getbands()[0]=='L')
    
    def image_rgb2lab(self):
        rgb2xyz = (0.412453, 0.357580, 0.180423, 0,
                   0.212671, 0.715160, 0.072169, 0,
                   0.019334, 0.119193, 0.950227, 0)
        self.image = self.image.convert('RGB', rgb2xyz)
        
    def image_lab2rgb(im):
        lab2rgb = ( 2.9515373,-1.2894116,-0.4738445,0,
                   -1.0851093, 1.9908566, 0.0372026,0,
                    0.0854934,-0.2694964, 1.0912975,0)
        self.image = self.image.convert('RGB',lab2rgb)
    
    def tuple_rgb2lab(self,tuple_):
        r, g, b = tuple_
        rgb = sRGBColor(r, g, b, is_upscaled=True)
        return convert_color(rgb,LabColor).get_value_tuple()
    
    def tuple_lab2rgb(self,tuple_):
        l, a, b = tuple_
        lab = LabColor(l, a, b)
        return convert_color(lab, sRGBColor).get_upscaled_value_tuple()
    
    def slicing(self,serie,n):
        return [list(serie[i*n:(i+1)*n])for i in range(n)]
    
    def save_ndarray(self,ndarray):
        np.save(self.name+'.npy',ndarray)
    
    def rgb_image2lab_ndarray(self):
        data = pd.DataFrame()
        data['RGB'] = list(self.image.getdata())
        data['Lab'] = data['RGB'].apply(num_rgb2lab)
        self.save_ndarray(self.series2ndarray(data['Lab']))
    
    def series2ndarray(self,data):
        size = int(math.sqrt(len(data)))
        data = pd.DataFrame(list(data))
        data_0 = self.slicing(data[0],size)
        data_1 = self.slicing(data[1],size)
        data_2 = self.slicing(data[2],size)
        data = np.array([data_0,data_1,data_2])
        return np.ndarray(shape=(3,size,size),buffer=data,dtype=float)
    
    def load_ndarray(self,name):
        return np.load(name)
    



In [322]:
    def grayscale(self):
        return (self.image.getbands()[0]=='L')

In [323]:
    def cut(self):
        size = min(self.size)
        self.image = self.image.crop((0,0,size,size))

In [324]:
    def resize(self):
        self.image = self.image.resize((128,128),Image.ANTIALIAS)

In [325]:
    #Função para mostrar as imagens
    def show_image(self):
        plt.imshow(self.image)
        plt.xticks([])
        plt.yticks([])


In [333]:
if __name__ == "__main__":
    im = pre_processing('clr_t1.jpeg')
    

In [11]:
def image_rgb2lab(im):
    rgb2xyz = (0.412453, 0.357580, 0.180423, 0,
               0.212671, 0.715160, 0.072169, 0,
               0.019334, 0.119193, 0.950227, 0)
    return im.convert('RGB', rgb2xyz)
    #return ImageCms.buildTransform(im, ImageCms.createProfile("sRGB"), ImageCms.createProfile("LAB"))

In [12]:
def image_lab2rgb(im):
    lab2rgb = ( 2.9515373,-1.2894116,-0.4738445,0,
               -1.0851093, 1.9908566, 0.0372026,0,
                0.0854934,-0.2694964, 1.0912975,0)
    return im.convert('RGB',lab2rgb)

In [178]:
def tuple_rgb2lab(tuple_):
    r, g, b = tuple_
    rgb = sRGBColor(r, g, b, is_upscaled=True)
    return convert_color(rgb,LabColor).get_value_tuple()

In [179]:
def slicing(serie,n):
    return [list(serie[i*n:(i+1)*n])for i in range(n)]

In [180]:
def tuple_lab2rgb(tuple_):
    l, a, b = tuple_
    lab = LabColor(l, a, b)
    return convert_color(lab, sRGBColor).get_upscaled_value_tuple()

In [16]:
def np_arrayLab(data):
    rgbs = pd.DataFrame(list(data),columns=['L','a','b'])
    print(rgbs.head(10))
    n = int(math.sqrt(len(rgbs['L'])))
    return np.array([slicing(rgbs['L'],n),slicing(rgbs['a'],n),slicing(rgbs['b'],n)])

In [114]:
def rgb2lab_ndarray(im):
    nome = 'teste'
    data = pd.DataFrame(list(im.getdata()),columns=['RGB'])
    data['RGB'] = list(im.getdata())
    data['Lab'] = data['RGB'].apply(num_rgb2lab)
    array = np_arrayLab(data['Lab'])
    return array
    
    

In [115]:
def save_ndarray(ndarray):
    np.save('3dsave.npy',ndarray)

In [149]:
def rgb_image2lab_ndarray(im):
    data = pd.DataFrame()
    data['RGB'] = list(im.getdata())
    data['Lab'] = data['RGB'].apply(num_rgb2lab)
    #save
    return series2ndarray(data['Lab'])

In [150]:
def series2ndarray(data):
    size = int(math.sqrt(len(data)))
    data = pd.DataFrame(list(data))
    data_0 = slicing(data[0],size)
    data_1 = slicing(data[1],size)
    data_2 = slicing(data[2],size)
    data = np.array([data_0,data_1,data_2])
    return np.ndarray(shape=(3,size,size),buffer=data,dtype=float)

In [116]:
def load_ndarray(name):
    return np.load(name)

In [139]:
im = Image.open('clr_t1.jpeg')
im = resize(cut(im))

In [274]:
def ndarray2series(data):
    data_0 = np.asarray(data[0]).reshape(-1)
    data_1 = np.asarray(data[1]).reshape(-1)
    data_2 = np.asarray(data[2]).reshape(-1)
    data = pd.DataFrame()
    data['Lab'] = list(list(data_0, data_1,data_2))
    #print(data['Lab'])
    return data['Lab']

In [278]:
def lab_ndarray2rgb_image(array):
    #ndarray = load_ndarray(name)
    print(array)
    img = Image.fromarray(array, 'RGB')
    #img.save('my.png')
    img.show()

In [279]:
def array2PIL(arr, size):
    mode = 'RGB'
    arr = arr.reshape(arr.shape[0]*arr.shape[1], arr.shape[2])
    if len(arr[0]) == 3:
        arr = np.c_[arr, 255*np.ones((len(arr),1), np.uint8)]
    return Image.frombuffer(mode, size, arr.tostring(), 'raw', mode, 0, 1) 

In [280]:
lab_ndarray2rgb_image(rgb_image2lab_ndarray(im))

[[[0.05830464 0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.26148098 0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.09213212 0.         0.         ... 0.         0.         0.        ]
  [0.         0.      

In [ ]:
from PIL import Image
import numpy as np

w, h = 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[256, 256] = [255, 0, 0]
data[0][0][0] = 1
print(type(data))
img = Image.fromarray(data, 'RGB')
img.save('my.png')
img.show()

In [272]:
im = Image.open('my.png')
print(len(data[0][0]))
#print(list(im.getdata()))

3
